In [3]:
import json
import torch
import pandas as pd

from tqdm import tqdm
from pathlib import Path
from sklearn.metrics import classification_report
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM

### DATA

In [4]:
DATA_DIR = Path.cwd() / "am_reasoning" / "data_files"

In [118]:
pe_df = pd.read_csv((DATA_DIR / "pe_dataset.csv"), index_col=0)

In [119]:
pe_df

,split,title,argument_bound_1,argument_bound_2,argument_id,ac,label,essay
0,TRAIN,Should students be taught to compete or to coo...,503,575,1,we should attach more importance to cooperatio...,MajorClaim,Should students be taught to compete or to coo...
1,TRAIN,Should students be taught to compete or to coo...,591,714,2,"through cooperation, children can learn about ...",Claim,Should students be taught to compete or to coo...
2,TRAIN,Should students be taught to compete or to coo...,716,851,3,What we acquired from team work is not only ho...,Premise,Should students be taught to compete or to coo...
3,TRAIN,Should students be taught to compete or to coo...,853,1086,4,"During the process of cooperation, children ca...",Premise,Should students be taught to compete or to coo...
4,TRAIN,Should students be taught to compete or to coo...,1088,1191,5,All of these skills help them to get on well w...,Premise,Should students be taught to compete or to coo...
...,...,...,...,...,...,...,...,...
6084,TRAIN,Children should studying hard or playing sport...,1275,1339,11,indirectly they will learn how to socialize ea...,Premise,Children should studying hard or playing sport...
6085,TRAIN,Children should studying hard or playing sport...,1341,1388,12,That will make children getting lots of friends,Premise,Children should studying hard or playing sport...
6086,TRAIN,Children should studying hard or playing sport...,1393,1436,13,they can contribute positively to community,Premise,Children should studying hard or playing sport...
6087,TRAIN,Children should studying hard or playing sport...,1448,1525,14,playing sport makes children getting healthy a...,Premise,Children should studying hard or playing sport...


In [120]:
pe_title_df = pe_df.groupby(['title', 'split'], sort=False).agg({
    'argument_bound_1': list,
    'argument_bound_2': list,
    'argument_id': list,
    'ac': list,
    'label': list,
    'essay': list    
}).reset_index()

In [121]:
pe_title_df

,title,split,argument_bound_1,argument_bound_2,argument_id,ac,label,essay
0,Should students be taught to compete or to coo...,TRAIN,"[503, 591, 716, 853, 1088, 1212, 1332, 1387, 1...","[575, 714, 851, 1086, 1191, 1301, 1376, 1492, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",[we should attach more importance to cooperati...,"[MajorClaim, Claim, Premise, Premise, Premise,...",[Should students be taught to compete or to co...
1,More people are migrating to other countries t...,TRAIN,"[391, 500, 626, 841, 948, 1089, 1168, 1209, 13...","[489, 624, 839, 946, 1057, 1156, 1204, 1307, 1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]",[they are able to sustain their cultural ident...,"[MajorClaim, Premise, Premise, Premise, Premis...",[More people are migrating to other countries ...
2,International tourism is now more common than ...,TRAIN,"[352, 487, 615, 786, 955, 1024, 1151, 1331, 14...","[475, 613, 784, 943, 1012, 1135, 1329, 1446, 1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",[it has contributed to the economic developmen...,"[MajorClaim, Premise, Premise, Premise, Claim,...",[International tourism is now more common than...
3,International tourism is now more common than ...,TEST,"[179, 262, 417, 532, 820, 953, 1074, 1160, 143...","[239, 376, 530, 818, 924, 1031, 1158, 1413, 15...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",[the tourism bring large profit for the destin...,"[Claim, MajorClaim, Premise, Premise, Premise,...",[International tourism is now more common than...
4,Living and studying overseas,TEST,"[286, 388, 566, 775, 885, 993, 1090, 1156, 124...","[359, 564, 765, 883, 988, 1078, 1148, 1238, 13...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]",[one who studies overseas will gain many skill...,"[MajorClaim, Claim, Premise, Claim, Premise, P...",[Living and studying overseas\n\nIt is every s...
...,...,...,...,...,...,...,...,...
392,Modern technology has impacted the traditional...,TRAIN,"[297, 415, 490, 713, 794, 1051, 1186, 1297, 14...","[413, 488, 702, 778, 1049, 1137, 1295, 1414, 1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",[the affect of technology on culture is undeni...,"[MajorClaim, Claim, Premise, Premise, Premise,...",[Modern technology has impacted the traditiona...
393,We can not forcedly put the same numbers of ma...,TEST,"[254, 333, 409, 557, 647, 778, 915, 1050, 1169...","[318, 407, 541, 645, 767, 897, 1035, 1163, 124...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[many different characters exist between male ...,"[Claim, Claim, Premise, Premise, Premise, Prem...",[We can not forcedly put the same numbers of m...
394,"Drugs, alcohol and messy sex lives",TRAIN,"[367, 431, 527, 661, 750, 787, 966, 1086, 1177...","[429, 525, 645, 737, 785, 951, 1073, 1172, 130...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",[this is a worrying tread which has negative i...,"[MajorClaim, Premise, Claim, Premise, Premise,...","[Drugs, alcohol and messy sex lives\n\nCelebri..."
395,Fatherhood should be as present as motherhood ...,TRAIN,"[311, 386, 502, 622, 818, 954, 1103, 1289, 139...","[384, 488, 607, 815, 937, 1087, 1278, 1370, 14...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",[fatherhood is a as vital part of a healty par...,"[MajorClaim, Claim, Premise, Premise, Claim, P...",[Fatherhood should be as present as motherhood...


In [122]:
len(pe_title_df.title.unique().tolist())

395

In [123]:
df_para = pd.read_csv((DATA_DIR / "PE_data.csv"))

In [124]:
df_para

,essay_id,para_id,para_types,para_text,adu_spans,ac_spans,ai_spans,AC_types,AR_pairs,AR_types
0,0,0,prompt,<prompt> Should students be taught to compete ...,[],[],[],[],[],[]
1,0,1,intro,<para-intro> It is always said that competitio...,"[(76, 97)]","[(86, 97)]","[(76, 85)]",['MajorClaim'],[],[]
2,0,2,body,"<para-body> First of all , <AC> through cooper...","[(1, 25), (26, 55), (56, 99), (100, 123)]","[(5, 25), (27, 55), (57, 99), (101, 123)]","[(1, 4), (26, 26), (56, 56), (100, 100)]","['Claim', 'Premise', 'Premise', 'Premise']","[(0, 1), (0, 2), (0, 3)]","['Support', 'Support', 'Support']"
3,0,3,body,"<para-body> On the other hand , <AC> the signi...","[(1, 22), (24, 37), (39, 63), (76, 139), (155,...","[(6, 22), (30, 37), (41, 63), (77, 139), (156,...","[(1, 5), (24, 29), (39, 40), (76, 76), (155, 1...","['Premise', 'Claim', 'Premise', 'Premise', 'Cl...","[(1, 0), (4, 2), (4, 3)]","['Support', 'Support', 'Support']"
4,0,4,conclusion,"<para-conclusion> Consequently , no matter fro...","[(1, 40)]","[(25, 40)]","[(1, 24)]",['MajorClaim'],[],[]
...,...,...,...,...,...,...,...,...,...,...
2230,401,2230,prompt,<prompt> A greater proportion of the budget sh...,[],[],[],[],[],[]
2231,401,2231,intro,"<para-intro> In today ' s world , the concept ...","[(26, 51)]","[(33, 51)]","[(26, 32)]",['MajorClaim'],[],[]
2232,401,2232,body,<para-body> <AC> The first reason why educatio...,"[(0, 22), (24, 56), (58, 71), (72, 98)]","[(1, 22), (27, 56), (60, 71), (74, 98)]","[(0, 0), (24, 26), (58, 59), (72, 73)]","['Premise', 'Premise', 'Claim', 'Premise']","[(2, 0), (2, 1), (2, 3)]","['Support', 'Support', 'Support']"
2233,401,2233,body,<para-body> The second reason why <AC> governm...,"[(1, 25), (27, 53), (55, 88), (90, 128)]","[(5, 25), (31, 53), (57, 88), (93, 128)]","[(1, 4), (27, 30), (55, 56), (90, 92)]","['Claim', 'Premise', 'Premise', 'Premise']","[(2, 1), (0, 2), (2, 3)]","['Support', 'Support', 'Support']"


In [125]:
def process_prompt_titles(row):
    
    if row.para_types != "prompt":
        return row.para_text
    else:
        return row.para_text.replace("<prompt> ", "").replace(" </prompt>", "")

In [126]:
df_para['essay_title'] = df_para.apply(lambda row: process_prompt_titles(row), axis=1)

In [127]:
df_para

,essay_id,para_id,para_types,para_text,adu_spans,ac_spans,ai_spans,AC_types,AR_pairs,AR_types,essay_title
0,0,0,prompt,<prompt> Should students be taught to compete ...,[],[],[],[],[],[],Should students be taught to compete or to coo...
1,0,1,intro,<para-intro> It is always said that competitio...,"[(76, 97)]","[(86, 97)]","[(76, 85)]",['MajorClaim'],[],[],<para-intro> It is always said that competitio...
2,0,2,body,"<para-body> First of all , <AC> through cooper...","[(1, 25), (26, 55), (56, 99), (100, 123)]","[(5, 25), (27, 55), (57, 99), (101, 123)]","[(1, 4), (26, 26), (56, 56), (100, 100)]","['Claim', 'Premise', 'Premise', 'Premise']","[(0, 1), (0, 2), (0, 3)]","['Support', 'Support', 'Support']","<para-body> First of all , <AC> through cooper..."
3,0,3,body,"<para-body> On the other hand , <AC> the signi...","[(1, 22), (24, 37), (39, 63), (76, 139), (155,...","[(6, 22), (30, 37), (41, 63), (77, 139), (156,...","[(1, 5), (24, 29), (39, 40), (76, 76), (155, 1...","['Premise', 'Claim', 'Premise', 'Premise', 'Cl...","[(1, 0), (4, 2), (4, 3)]","['Support', 'Support', 'Support']","<para-body> On the other hand , <AC> the signi..."
4,0,4,conclusion,"<para-conclusion> Consequently , no matter fro...","[(1, 40)]","[(25, 40)]","[(1, 24)]",['MajorClaim'],[],[],"<para-conclusion> Consequently , no matter fro..."
...,...,...,...,...,...,...,...,...,...,...,...
2230,401,2230,prompt,<prompt> A greater proportion of the budget sh...,[],[],[],[],[],[],A greater proportion of the budget should be a...
2231,401,2231,intro,"<para-intro> In today ' s world , the concept ...","[(26, 51)]","[(33, 51)]","[(26, 32)]",['MajorClaim'],[],[],"<para-intro> In today ' s world , the concept ..."
2232,401,2232,body,<para-body> <AC> The first reason why educatio...,"[(0, 22), (24, 56), (58, 71), (72, 98)]","[(1, 22), (27, 56), (60, 71), (74, 98)]","[(0, 0), (24, 26), (58, 59), (72, 73)]","['Premise', 'Premise', 'Claim', 'Premise']","[(2, 0), (2, 1), (2, 3)]","['Support', 'Support', 'Support']",<para-body> <AC> The first reason why educatio...
2233,401,2233,body,<para-body> The second reason why <AC> governm...,"[(1, 25), (27, 53), (55, 88), (90, 128)]","[(5, 25), (31, 53), (57, 88), (93, 128)]","[(1, 4), (27, 30), (55, 56), (90, 92)]","['Claim', 'Premise', 'Premise', 'Premise']","[(2, 1), (0, 2), (2, 3)]","['Support', 'Support', 'Support']",<para-body> The second reason why <AC> governm...


In [128]:
pe_para_df = df_para.groupby(['essay_id']).agg({
    'AC_types': list,
    'AR_pairs': list,
    'AR_types': list,
    'essay_title': list,  
}).reset_index()

In [129]:
pe_para_df

,essay_id,AC_types,AR_pairs,AR_types,essay_title
0,0,"[[], ['MajorClaim'], ['Claim', 'Premise', 'Pre...","[[], [], [(0, 1), (0, 2), (0, 3)], [(1, 0), (4...","[[], [], ['Support', 'Support', 'Support'], ['...",[Should students be taught to compete or to co...
1,1,"[[], ['MajorClaim'], ['Premise', 'Premise', 'P...","[[], [], [(4, 0), (4, 1), (4, 2), (4, 3)], [(5...","[[], [], ['Support', 'Support', 'Support', 'Su...",[More people are migrating to other countries ...
2,2,"[[], ['MajorClaim'], ['Premise', 'Premise', 'P...","[[], [], [(3, 0), (3, 1), (3, 2)], [(3, 0), (3...","[[], [], ['Support', 'Support', 'Support'], ['...",[International tourism is now more common than...
3,3,"[[], ['MajorClaim'], ['Premise', 'Premise', 'P...","[[], [], [(3, 0), (3, 1), (3, 2)], [(3, 0), (3...","[[], [], ['Support', 'Support', 'Support'], ['...","[Will newspapers become a thing of the past ?,..."
4,4,"[[], ['MajorClaim'], ['Premise', 'Premise', 'P...","[[], [], [(3, 0), (3, 1), (3, 2)], [(5, 0), (5...","[[], [], ['Support', 'Support', 'Support'], ['...","[Government budget focus , young children or u..."
...,...,...,...,...,...
397,397,"[[], ['MajorClaim'], ['Claim', 'Premise', 'Pre...","[[], [], [(0, 1), (0, 2)], [(0, 1), (0, 2)], []]","[[], [], ['Support', 'Support'], ['Support', '...",[The maintenance of traditional skills and way...
398,398,"[[], [], ['Claim', 'Premise', 'Premise', 'Prem...","[[], [], [(0, 1), (0, 2), (0, 3)], [(1, 0)], [...","[[], [], ['Support', 'Support', 'Support'], ['...","[University education restriction, <para-intro..."
399,399,"[[], ['MajorClaim'], ['Premise', 'Premise', 'P...","[[], [], [(4, 0), (2, 1), (4, 2), (4, 3)], [(1...","[[], [], ['Support', 'Support', 'Support', 'Su...",[Police force carries guns - significant impac...
400,400,"[[], ['MajorClaim'], ['Claim', 'Premise', 'Pre...","[[], [], [(0, 1), (0, 2)], [(0, 1), (0, 2)], [...","[[], [], ['Support', 'Attack'], ['Support', 'S...","[Gun control and increasing violence, <para-in..."


In [130]:
import ast

In [131]:
def process_para_df(row, column): 
    
    if column == "AC_types":
        row.AC_types = [ast.literal_eval(elem) for elem in row.AC_types]
        return [elem for elem in row.AC_types if elem]
    elif column == "AR_pairs":
        row.AR_pairs = [ast.literal_eval(elem) for elem in row.AR_pairs]
        return [elem for elem in row.AR_pairs if elem]
    elif column == "AR_types":
        row.AR_types = [ast.literal_eval(elem) for elem in row.AR_types]
        return [elem for elem in row.AR_types if elem]
    elif column == "essay_title":
        return row.essay_title[0]

In [132]:
pe_para_df['AC_types'] = pe_para_df.apply(lambda row: process_para_df(row, "AC_types"), axis=1) # type: ignore
pe_para_df['AR_pairs'] = pe_para_df.apply(lambda row: process_para_df(row, "AR_pairs"), axis=1) # type: ignore
pe_para_df['AR_types'] = pe_para_df.apply(lambda row: process_para_df(row, "AR_types"), axis=1) # type: ignore
pe_para_df['essay_title'] = pe_para_df.apply(lambda row: process_para_df(row, "essay_title"), axis=1) # type: ignore

In [133]:
pe_para_df

,essay_id,AC_types,AR_pairs,AR_types,essay_title
0,0,"[[MajorClaim], [Claim, Premise, Premise, Premi...","[[(0, 1), (0, 2), (0, 3)], [(1, 0), (4, 2), (4...","[[Support, Support, Support], [Support, Suppor...",Should students be taught to compete or to coo...
1,1,"[[MajorClaim], [Premise, Premise, Premise, Pre...","[[(4, 0), (4, 1), (4, 2), (4, 3)], [(5, 0), (0...","[[Support, Support, Support, Support], [Suppor...",More people are migrating to other countries t...
2,2,"[[MajorClaim], [Premise, Premise, Premise, Cla...","[[(3, 0), (3, 1), (3, 2)], [(3, 0), (3, 1), (3...","[[Support, Support, Support], [Support, Suppor...",International tourism is now more common than ...
3,3,"[[MajorClaim], [Premise, Premise, Premise, Cla...","[[(3, 0), (3, 1), (3, 2)], [(3, 0), (3, 1), (3...","[[Support, Support, Support], [Support, Suppor...",Will newspapers become a thing of the past ?
4,4,"[[MajorClaim], [Premise, Premise, Premise, Cla...","[[(3, 0), (3, 1), (3, 2)], [(5, 0), (5, 1), (5...","[[Support, Support, Support], [Support, Suppor...","Government budget focus , young children or un..."
...,...,...,...,...,...
397,397,"[[MajorClaim], [Claim, Premise, Premise], [Cla...","[[(0, 1), (0, 2)], [(0, 1), (0, 2)]]","[[Support, Support], [Support, Support]]",The maintenance of traditional skills and ways...
398,398,"[[Claim, Premise, Premise, Premise], [Premise,...","[[(0, 1), (0, 2), (0, 3)], [(1, 0)], [(0, 1)],...","[[Support, Support, Support], [Support], [Supp...",University education restriction
399,399,"[[MajorClaim], [Premise, Premise, Premise, Pre...","[[(4, 0), (2, 1), (4, 2), (4, 3)], [(1, 0), (2...","[[Support, Support, Support, Support], [Suppor...",Police force carries guns - significant impact...
400,400,"[[MajorClaim], [Claim, Premise, Premise], [Cla...","[[(0, 1), (0, 2)], [(0, 1), (0, 2)], [(1, 0), ...","[[Support, Attack], [Support, Support], [Suppo...",Gun control and increasing violence


In [134]:
pe_title_df

,title,split,argument_bound_1,argument_bound_2,argument_id,ac,label,essay
0,Should students be taught to compete or to coo...,TRAIN,"[503, 591, 716, 853, 1088, 1212, 1332, 1387, 1...","[575, 714, 851, 1086, 1191, 1301, 1376, 1492, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",[we should attach more importance to cooperati...,"[MajorClaim, Claim, Premise, Premise, Premise,...",[Should students be taught to compete or to co...
1,More people are migrating to other countries t...,TRAIN,"[391, 500, 626, 841, 948, 1089, 1168, 1209, 13...","[489, 624, 839, 946, 1057, 1156, 1204, 1307, 1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]",[they are able to sustain their cultural ident...,"[MajorClaim, Premise, Premise, Premise, Premis...",[More people are migrating to other countries ...
2,International tourism is now more common than ...,TRAIN,"[352, 487, 615, 786, 955, 1024, 1151, 1331, 14...","[475, 613, 784, 943, 1012, 1135, 1329, 1446, 1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",[it has contributed to the economic developmen...,"[MajorClaim, Premise, Premise, Premise, Claim,...",[International tourism is now more common than...
3,International tourism is now more common than ...,TEST,"[179, 262, 417, 532, 820, 953, 1074, 1160, 143...","[239, 376, 530, 818, 924, 1031, 1158, 1413, 15...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",[the tourism bring large profit for the destin...,"[Claim, MajorClaim, Premise, Premise, Premise,...",[International tourism is now more common than...
4,Living and studying overseas,TEST,"[286, 388, 566, 775, 885, 993, 1090, 1156, 124...","[359, 564, 765, 883, 988, 1078, 1148, 1238, 13...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]",[one who studies overseas will gain many skill...,"[MajorClaim, Claim, Premise, Claim, Premise, P...",[Living and studying overseas\n\nIt is every s...
...,...,...,...,...,...,...,...,...
392,Modern technology has impacted the traditional...,TRAIN,"[297, 415, 490, 713, 794, 1051, 1186, 1297, 14...","[413, 488, 702, 778, 1049, 1137, 1295, 1414, 1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",[the affect of technology on culture is undeni...,"[MajorClaim, Claim, Premise, Premise, Premise,...",[Modern technology has impacted the traditiona...
393,We can not forcedly put the same numbers of ma...,TEST,"[254, 333, 409, 557, 647, 778, 915, 1050, 1169...","[318, 407, 541, 645, 767, 897, 1035, 1163, 124...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",[many different characters exist between male ...,"[Claim, Claim, Premise, Premise, Premise, Prem...",[We can not forcedly put the same numbers of m...
394,"Drugs, alcohol and messy sex lives",TRAIN,"[367, 431, 527, 661, 750, 787, 966, 1086, 1177...","[429, 525, 645, 737, 785, 951, 1073, 1172, 130...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",[this is a worrying tread which has negative i...,"[MajorClaim, Premise, Claim, Premise, Premise,...","[Drugs, alcohol and messy sex lives\n\nCelebri..."
395,Fatherhood should be as present as motherhood ...,TRAIN,"[311, 386, 502, 622, 818, 954, 1103, 1289, 139...","[384, 488, 607, 815, 937, 1087, 1278, 1370, 14...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]",[fatherhood is a as vital part of a healty par...,"[MajorClaim, Claim, Premise, Premise, Claim, P...",[Fatherhood should be as present as motherhood...


In [135]:
pe_title_df.iloc[0]['title'], pe_para_df.iloc[0]['essay_title']

('Should students be taught to compete or to cooperate?',
 'Should students be taught to compete or to cooperate ?')

In [117]:
pe_para_df['essay_title'].tolist()

['Should students be taught to compete or to cooperate ?',
 'More people are migrating to other countries than ever before',
 'International tourism is now more common than ever before',
 'Will newspapers become a thing of the past ?',
 'Government budget focus , young children or university ?',
 'Roommates quality and their importance',
 'Should governments spend more money on improving roads and highways',
 'Physical exercise',
 "Technology inspires children ' s creativity instead of weakening it",
 'Lessons with teachers versus others sources',
 'Email and text messaging threats written language',
 'Using animals for the benefit of the human beings',
 'Television is the culprit for destroying communication between friends & family',
 'Gender Equality at university admission',
 'Some young adults want independence from their parents quickly',
 'Air pollution',
 'Young people should go to university or not',
 'Effects of mobile phones',
 'Influence of large factories',
 'Prepared Food

In [111]:
merged_df = pd.merge(pe_title_df, pe_para_df, left_on="title", right_on="essay_title", how="outer")

In [112]:
merged_df

,title,split,argument_bound_1,argument_bound_2,argument_id,ac,label,essay,essay_id,AC_types,AR_pairs,AR_types,essay_title
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,"[[Claim, Premise, Premise, Claim, Premise, Pre...","[[(0, 1), (0, 2), (3, 4), (6, 5), (3, 6)], [(0...","[[Support, Support, Support, Support, Support]...",""" Pen and Paper "" Examination"
1,"""Pen and Paper"" Examination",TRAIN,"[349, 388, 473, 656, 688, 806, 900, 1050, 1133...","[386, 471, 645, 686, 791, 884, 1048, 1112, 120...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[There are clearly advantages to exams, They h...","[Claim, Premise, Premise, Claim, Premise, Prem...","[""Pen and Paper"" Examination\n\nExaminations a...",NaN,NaN,NaN,NaN,NaN
2,A greater proportion of the budget should be a...,TRAIN,"[274, 541, 736, 805, 1112, 1233, 1403, 1619, 2...","[387, 723, 796, 944, 1220, 1388, 1602, 1812, 3...","[1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, ...",[governments should devote a greater portion o...,"[MajorClaim, Premise, Claim, Premise, Premise,...",[A greater proportion of the budget should be ...,376.0,"[[MajorClaim], [Premise, Claim, Premise], [Pre...","[[(1, 0), (1, 2)], [(1, 0), (1, 2)]]","[[Support, Support], [Support, Support]]",A greater proportion of the budget should be a...
3,A greater proportion of the budget should be a...,TRAIN,"[274, 541, 736, 805, 1112, 1233, 1403, 1619, 2...","[387, 723, 796, 944, 1220, 1388, 1602, 1812, 3...","[1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, ...",[governments should devote a greater portion o...,"[MajorClaim, Premise, Claim, Premise, Premise,...",[A greater proportion of the budget should be ...,381.0,"[[MajorClaim], [Premise, Premise, Claim], [Cla...","[[(2, 0), (2, 1)], [(0, 1), (0, 2), (0, 3)]]","[[Support, Support], [Support, Support, Support]]",A greater proportion of the budget should be a...
4,A greater proportion of the budget should be a...,TRAIN,"[274, 541, 736, 805, 1112, 1233, 1403, 1619, 2...","[387, 723, 796, 944, 1220, 1388, 1602, 1812, 3...","[1, 2, 3, 4, 5, 6, 7, 8, 1, 2, 3, 4, 5, 6, 7, ...",[governments should devote a greater portion o...,"[MajorClaim, Premise, Claim, Premise, Premise,...",[A greater proportion of the budget should be ...,401.0,"[[MajorClaim], [Premise, Premise, Claim, Premi...","[[(2, 0), (2, 1), (2, 3)], [(2, 1), (0, 2), (2...","[[Support, Support, Support], [Support, Suppor...",A greater proportion of the budget should be a...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,Zoos have no useful purpose?,TRAIN,"[365, 473, 594, 782, 876, 1040, 1153, 1280, 14...","[471, 592, 780, 874, 1037, 1151, 1278, 1441, 1...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[zoos provide social interaction, education, a...","[MajorClaim, Premise, Premise, Premise, Claim,...",[Zoos have no useful purpose?\n\nHave you ever...,NaN,NaN,NaN,NaN,NaN
550,Zoos should be banned,TEST,"[332, 446, 645, 704, 768, 836, 932, 1039, 1136...","[419, 630, 700, 766, 834, 916, 1037, 1134, 133...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]",[zoos have irreplaceable benefits in terms of ...,"[MajorClaim, Premise, Claim, Premise, Premise,...",[Zoos should be banned\n\nChildren tend to vis...,296.0,"[[MajorClaim], [Premise, Claim, Premise, Premi...","[[(1, 0), (1, 2), (1, 3), (1, 4)], [(2, 1), (0...","[[Support, Support, Support, Support], [Suppor...",Zoos should be banned
551,Zoos should be built to protect rural animals,TRAIN,"[353, 425, 506, 643, 745, 861, 1032, 1107, 123...","[408, 504, 641, 731, 849, 1017, 1091, 1191, 13...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]",[animals should live in natural habitats inste...,"[MajorClaim, Premise, Premise, Claim, Premise,...",[Zoos should be built to protect rural animals...,37.0,"[[MajorClaim], [Premise, Premise, Claim, Premi...","[[(2, 0), (2, 1), (6, 3), (6, 4), (6, 5)], [(0...","[[Support, Support, Support, Support, Support]...",Zoos should be built to protect rural animals
552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,"[[MajorClaim], [C

In [ ]:
def get_full_essay_am(row):
    
    title = row.title
    acs_l = row.ac
    labels_l = row.labels
    essay = row.essay
    
    

### Title embeddings

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
embedding_model = AutoModel.from_pretrained("google-bert/bert-base-uncased").to(device)

### Tokenizer and Model

In [2]:
model_id = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

In [5]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": [
      "lm_head",
   

### Data

In [6]:
DATA_DIR = Path.cwd() / "am_reasoning" / "data_files"

In [1]:
df = pd.read_csv((DATA_DIR / "pe_dataset.csv"), index_col=0)

NameError: name 'pd' is not defined

In [8]:
df.columns

Index(['split', 'title', 'argument_bound_1', 'argument_bound_2', 'argument_id',
       'ac', 'label', 'essay'],
      dtype='object')

In [9]:
df.shape

(6089, 8)

In [10]:
df.split.value_counts()

split
TRAIN    4823
TEST     1266
Name: count, dtype: int64

### Template Messages

In [11]:
# sys_msg = """You are an expert in argumentation analysis. You will be given a complete argument. Your task is to analyze the argument holistically and classify it as exactly ONE of the following categories: Position, Claim, or Premise. 

# First, think through your analysis step by step, considering the function and structure of the argument. Explain your reasoning, identifying key characteristics that point to a specific classification.

# Then, provide your final classification as a single word: Position, Claim, or Premise."""

In [12]:
# instruction = """### You are an expert in Argument Mining. You are given an argument component from an essay. Your task is to classify the argument component as either "MajorClaim", "Claim" or "Premise". You must return the argument component type in following JSON format:: {"component_type": [component_type (str)]}


# """

In [13]:
# instruction = """### You are an expert in Argument Mining and discourse analysis. 

# INPUT:
# - You are given an argument component from an essay. 
# - You are given the full text of the essay.

# TASK:
# - Your task is to classify the whole argument component, as ONLY ONE of: "MajorClaim", "Claim" or "Premise".
# - Explain your reasoning step-by-step before making your final classification.

# Then, return your final classification in the following JSON format:
# {"component_type": [Answer]}

# Example:

# {"component_type": ["Claim"]}

# """

In [15]:
reasons_d = {}

In [ ]:
instruction = """### You are an expert in Argument Mining and discourse analysis. 

INPUT: 
- You are given the full text of an argumentative essay on a topic.
- You are given the list of coherent argument units in the essay.

TASK:
- Your task is to classify the ENTIRE argument component as ONE of these three categories ONLY: "MajorClaim", "Claim" or "Premise".
- Do NOT break down the argument component into smaller parts with different classifications.
- The WHOLE component must receive exactly ONE classification.

REASONING PROCESS:
First, analyze the component considering these factors:
1. Semantic content: What is the meaning and purpose of this component in the argument?
2. Syntactic structure: How is this component phrased? (declarative statement, supporting evidence, etc.)
3. Context: How does this component relate to the overall essay thesis?
4. Function: 
   - Does it express the author's central position on the main topic? (MajorClaim)
   - Does it express a specific stance that supports or develops the main position? (Claim)
   - Does it provide evidence, examples, or justification for a claim? (Premise)

Provide your step-by-step reasoning, then classify the ENTIRE argument component as exactly ONE of: "MajorClaim", "Claim", or "Premise".

Finally, return your classification in the following JSON format:
{"component_type": ["Your Answer"]}

Example:
{"component_type": ["Claim"]}

"""

In [50]:
df_s = df.sample(100).reset_index(drop=True)

In [51]:
sys_msgs = []
user_msgs = []

for _, row in df_s.iterrows():

    sys_msg = {"role": "system", "content": instruction}
    #usr_msg = {"role": "user", "content": f"""### Here is the argument component to be classified: {row['ac']}"""} 
    
    #msg_c = f"""### Here is the complete essay text: {df.iloc[0]['essay']}""" + "\n\n" + f"""### Here is the argument component to be classified: {row['ac']}"""
    
    usr_msg = {"role": "user", "content": f"""### Here is the complete essay text: {row['essay']}""" + "\n\n" + f"""### Here is the argument component to be classified: {row['ac']}"""}
    
    
    sys_msgs.append(sys_msg)
    user_msgs.append(usr_msg)    
    

In [52]:
messages = []

for i in range(len(sys_msgs)):
    messages.append([sys_msgs[i], user_msgs[i]])

In [53]:
len(messages)

100

In [54]:
messages[5][1]['content'].split("###")[2]

' Here is the argument component to be classified: the benefit of free to access to the classes eclipse the issue of lower points'

In [55]:
#print(usr_msg)

In [56]:
# messages = [
#     {"role": "system", "content": instruction},
#     {"role": "user", "content": usr_msg},
# ]

In [57]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [58]:
for i, message in tqdm(enumerate(messages), "generating CoT", len(messages)):

    input_ids = tokenizer.apply_chat_template(message, add_generation_prompt=True, return_tensors="pt").to(model.device) # type: ignore
    outputs = model.generate(input_ids, max_new_tokens=1024, eos_token_id=terminators, do_sample=True, temperature=0.1, top_p=0.9)
    
    response = outputs[0][input_ids.shape[-1]:]
    reasons_d[message[1]['content'].split("###")[2]] = tokenizer.decode(response, skip_special_tokens=True)

generating CoT: 100%|██████████| 100/100 [21:57<00:00, 13.18s/it]


In [59]:
reasons_d

{' Here is the argument component to be classified: This has gotten worse with twitter, myspace, and facebook': 'Okay, so I need to classify the argument component "This has gotten worse with twitter, myspace, and facebook" as either MajorClaim, Claim, or Premise. Let me break this down step by step.\n\nFirst, I\'ll look at the semantic content. The component is stating that the situation has worsened due to specific platforms. It\'s pointing out a negative impact of technology on language, specifically how social media platforms are contributing to this issue. So, it\'s about the effect of these platforms on language use.\n\nNext, syntactic structure. It\'s a declarative statement, but more importantly, it\'s a specific example supporting the main point. It\'s not just a general statement; it\'s tied to the examples given.\n\nLooking at the context, the essay\'s thesis is that technology designed to simplify life often complicates it. The component is providing evidence that technolog

In [60]:
reasons_l = [v.split("\n\n") for k,v in reasons_d.items()]

In [61]:
reasons_l

[['Okay, so I need to classify the argument component "This has gotten worse with twitter, myspace, and facebook" as either MajorClaim, Claim, or Premise. Let me break this down step by step.',
  "First, I'll look at the semantic content. The component is stating that the situation has worsened due to specific platforms. It's pointing out a negative impact of technology on language, specifically how social media platforms are contributing to this issue. So, it's about the effect of these platforms on language use.",
  "Next, syntactic structure. It's a declarative statement, but more importantly, it's a specific example supporting the main point. It's not just a general statement; it's tied to the examples given.",
  "Looking at the context, the essay's thesis is that technology designed to simplify life often complicates it. The component is providing evidence that technology, specifically social media, is making language worse, which supports the thesis by showing a negative side eff

In [62]:
reasoning_chains = []
classifications = []

for chain in reasons_l:
    reasoning_chains.append(chain[:-1])
    classifications.append(chain[-1])

In [63]:
reasoning_chains

[['Okay, so I need to classify the argument component "This has gotten worse with twitter, myspace, and facebook" as either MajorClaim, Claim, or Premise. Let me break this down step by step.',
  "First, I'll look at the semantic content. The component is stating that the situation has worsened due to specific platforms. It's pointing out a negative impact of technology on language, specifically how social media platforms are contributing to this issue. So, it's about the effect of these platforms on language use.",
  "Next, syntactic structure. It's a declarative statement, but more importantly, it's a specific example supporting the main point. It's not just a general statement; it's tied to the examples given.",
  "Looking at the context, the essay's thesis is that technology designed to simplify life often complicates it. The component is providing evidence that technology, specifically social media, is making language worse, which supports the thesis by showing a negative side eff

In [64]:
classifications

['{"component_type": ["Premise"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["MajorClaim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["MajorClaim"]}',
 '{"component_type": ["MajorClaim"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["MajorClaim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Premise"]}',
 '{"comp

In [65]:
predictions = [json.loads(elem)['component_type'][0] for elem in classifications]

In [66]:
grounds = df_s.label.tolist()

In [ ]:
#predictions = predictions[20:]

In [69]:
len(grounds), len(predictions)

(100, 100)

In [72]:
print(classification_report(grounds, predictions, digits=3))

              precision    recall  f1-score   support

       Claim      0.350     0.467     0.400        30
  MajorClaim      0.462     0.750     0.571         8
     Premise      0.766     0.581     0.661        62

    accuracy                          0.560       100
   macro avg      0.526     0.599     0.544       100
weighted avg      0.617     0.560     0.575       100



In [258]:
# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(model.device) # type: ignore

In [259]:
#input_ids

In [260]:
#print(tokenizer.decode(input_ids[0]))

In [261]:
# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

In [262]:
# outputs = model.generate(
#     input_ids,
#     max_new_tokens=1024,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.1,
#     top_p=0.9,
# )

In [263]:
#response = outputs[0][input_ids.shape[-1]:]

In [264]:
#print(tokenizer.decode(response, skip_special_tokens=True))

In [265]:
#reasons_d[usr_msg] = tokenizer.decode(response, skip_special_tokens=True)

In [266]:
#reasons_d